In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import sys
import json

import numpy as np
import pandas as pd

import torch

import glob

from PIL import Image

from tqdm.auto import tqdm

from transformers import AutoTokenizer, AutoModel, CLIPImageProcessor

In [2]:
BATCH_SIZE = 64
MAX_LENGTH = 77
# MODEL_NAME = 'BAAI/bge-base-en-v1.5'
MODEL_NAME = 'openai/clip-vit-large-patch14'
# MODEL_NAME = 'laion/CLIP-ViT-bigG-14-laion2B-39B-b160k'
# MODEL_NAME = 'johngiorgi/declutr-base'
# MAX_LENGTH = 88
# MODEL_NAME = '../Llama-2-7b-hf'

# load data

In [3]:
INPUT_DIR = '../kcg-ml-image-pipeline/output/dataset/data/environmental/'
PMT_PATH = 'data/environmental/prompt.json'
EMB_PATH = 'data/environmental/clip_text_emb.npz'

## save json

In [5]:
sys.path.append(os.path.abspath('../kcg-ml-vae-test/'))
from utilities.utils import read_embedding_data, read_msg_pack

In [6]:
paths = sorted(glob.glob(os.path.join(INPUT_DIR, '**/*_data.msgpack')))

file_paths = list()
file_hashs = list()
positive_prompts = list()
negative_prompts = list()
creation_times = list()
for path in tqdm(paths):
    mp = read_msg_pack(path)
    file_paths.append(mp['file_path'])
    file_hashs.append(mp['file_hash'])
    positive_prompts.append(mp['positive_prompt'])
    negative_prompts.append(mp['negative_prompt'])
    creation_times.append(mp['creation_time'])

  0%|          | 0/75898 [00:00<?, ?it/s]

In [7]:
os.makedirs(os.path.split(PMT_PATH)[0], exist_ok=True)

json.dump(
    pd.DataFrame(zip(positive_prompts, negative_prompts, file_paths, creation_times), columns=['positive_prompt', 'negative_prompt', 'file_path', 'creation_time'], index=file_hashs).to_dict(orient='index'),
    open(PMT_PATH, 'w')
)

## load json

In [13]:
file_hashs = list()
file_paths = list()
positive_prompts = list()
negative_prompts = list()

for file_hash, info in json.load(open(PMT_PATH)).items():
    
    file_hashs.append(file_hash)
    file_paths.append(info['file_path'])
    positive_prompts.append(info['positive_prompt'])
    negative_prompts.append(info['negative_prompt'])

# load text embedder

## from kcg-ml-sd1p4

In [6]:
# !ln -s ../kcg-ml-sd1p4/input/ input
# !ln -s ../kcg-ml-sd1p4/output/ output

In [7]:
sys.path.append(os.path.abspath('../kcg-ml-sd1p4/'))
from stable_diffusion.model.clip_text_embedder import CLIPTextEmbedder

INFO: Created a temporary directory at /tmp/tmpejv92m2d
INFO: Writing /tmp/tmpejv92m2d/_remote_module_non_scriptable.py


In [10]:
clip_text_embedder = CLIPTextEmbedder(device='cuda')
_ = clip_text_embedder.load_submodels()

Loading tokenizer and transformer...[DONE]	2,403.54ms                             


In [11]:
def worker(texts):
    
    clip_text_opt = clip_text_embedder.forward_return_all(texts)

    last_hidden_state = clip_text_opt.last_hidden_state.detach().cpu().numpy()
    pooler_output = clip_text_opt.pooler_output.detach().cpu().numpy()
    attention_mask = clip_text_opt.attention_mask.detach().cpu().numpy()
    
    return last_hidden_state, pooler_output, attention_mask

## from transformers

In [14]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, local_files_only=True)

### from CLIP model

In [15]:
transformer = AutoModel.from_pretrained(MODEL_NAME, local_files_only=True).text_model.cuda().eval()

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [16]:
def worker(texts, use_penultimate=False):
    
    batch_encoding = tokenizer(
        texts,
        truncation=True, max_length=MAX_LENGTH, return_length=True,
        return_overflowing_tokens=False, padding="max_length", return_tensors="pt"
    )

    tokens = batch_encoding["input_ids"].cuda()

    clip_text_opt = transformer(input_ids=tokens, output_hidden_states=True, return_dict=True)
    
    attention_mask = batch_encoding.attention_mask.detach().cpu().numpy()
    pooler_output = clip_text_opt.pooler_output.detach().cpu().numpy()
    
    if use_penultimate:
        last_hidden_state = clip_text_opt.hidden_states[-1].detach().cpu().numpy()
    else:
        last_hidden_state = clip_text_opt.last_hidden_state.detach().cpu().numpy()
    
    return last_hidden_state, pooler_output, attention_mask

### from LLM model

In [9]:
def worker(texts):
    
    batch_encoding = tokenizer(
        texts,
        truncation=True, max_length=MAX_LENGTH, return_length=True,
        return_overflowing_tokens=False, padding="max_length", return_tensors="pt"
    )

    tokens = batch_encoding["input_ids"].cuda()

    clip_text_opt = transformer(input_ids=tokens)

    last_hidden_state = clip_text_opt.last_hidden_state.detach().cpu().numpy()
    attention_mask = batch_encoding.attention_mask.detach().cpu().numpy()
    
    pooler_output = clip_text_opt.pooler_output.detach().cpu().numpy()
    # pooler_output = None
    
    return last_hidden_state, pooler_output, attention_mask

In [10]:
transformer = AutoModel.from_pretrained(MODEL_NAME).cuda().eval()

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /johngiorgi/declutr-base/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fdb2e5ce7d0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 844b863d-ff1b-4646-a032-384b3cb8a040)')' thrown while requesting HEAD https://huggingface.co/johngiorgi/declutr-base/resolve/main/config.json
Some weights of the model checkpoint at johngiorgi/declutr-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expect

### from LLM model

In [8]:
tokenizer.pad_token = "[PAD]"
tokenizer.padding_side = "left"

transformer = AutoModel.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, load_in_8bit=True, device_map='auto').eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at ../Llama-2-7b-hf were not used when initializing LlamaModel: ['lm_head.weight']
- This IS expected if you are initializing LlamaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LlamaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# embed & save

In [19]:
positive_last_hidden_states = list()
positive_pooler_outputs = list()
positive_attention_masks = list()

negative_last_hidden_states = list()
negative_pooler_outputs = list()
negative_attention_masks = list()

with torch.no_grad():
    
    for i in tqdm(range(0, len(positive_prompts), BATCH_SIZE)):
        
        last_hidden_state, pooler_output, attention_mask = worker(positive_prompts[i:i+BATCH_SIZE])
        
        # positive_last_hidden_states.append(last_hidden_state)
        # positive_attention_masks.append(attention_mask)
        if pooler_output is not None:
            positive_pooler_outputs.append(pooler_output)
    
    for i in tqdm(range(0, len(negative_prompts), BATCH_SIZE)):
        
        last_hidden_state, pooler_output, attention_mask = worker(negative_prompts[i:i+BATCH_SIZE])
        
#         negative_last_hidden_states.append(last_hidden_state)
#         negative_attention_masks.append(attention_mask)
        if pooler_output is not None:
            negative_pooler_outputs.append(pooler_output)

  0%|          | 0/709 [00:00<?, ?it/s]

  0%|          | 0/709 [00:00<?, ?it/s]

In [20]:
# positive_last_hidden_states = np.concatenate(positive_last_hidden_states, axis=0)
# positive_attention_masks = np.concatenate(positive_attention_masks, axis=0)
if len(positive_pooler_outputs) > 0:
    positive_pooler_outputs = np.concatenate(positive_pooler_outputs, axis=0)

In [21]:
# negative_last_hidden_states = np.concatenate(negative_last_hidden_states, axis=0)
# negative_attention_masks = np.concatenate(negative_attention_masks, axis=0)
if len(positive_pooler_outputs) > 0:
    negative_pooler_outputs = np.concatenate(negative_pooler_outputs, axis=0)

In [22]:
os.makedirs(os.path.split(EMB_PATH)[0], exist_ok=True)

np.savez(
    EMB_PATH, 
    file_hashs=np.array(file_hashs), 
    file_paths=np.array(file_paths), 
    # positive_last_hidden_states=positive_last_hidden_states, 
    # positive_attention_masks=positive_attention_masks,
    positive_pooler_outputs=positive_pooler_outputs,
    # negative_last_hidden_states=negative_last_hidden_states,
    # negative_attention_masks=negative_attention_masks,
    negative_pooler_outputs=negative_pooler_outputs
)

# vision

In [3]:
# INPUT_DIR = '../dataset/civitai-stable-diffusion-337k/images/'
# OUTPUT_DIR = '../dataset/civitai-stable-diffusion-337k/clip/'

INPUT_DIR = '../dataset/scrap/steam/screenshot/'
OUTPUT_DIR = '../dataset/scrap/steam/clip/'

# INPUT_DIR = '../dataset/scrap/leonardo/images/'
# OUTPUT_DIR = '../dataset/scrap/leonardo/clip/'

# INPUT_DIR = '../dataset/midjourney-messages/images/'
# OUTPUT_DIR = '../dataset/midjourney-messages/clip/'

In [4]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [5]:
preprocessor = CLIPImageProcessor.from_pretrained(MODEL_NAME, local_files_only=True)

clip_model = AutoModel.from_pretrained(MODEL_NAME, local_files_only=True).cuda().eval()

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [8]:
file_names = list()

for file_name in os.listdir(INPUT_DIR):
    if not file_name.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')):
        continue
    clip_path = os.path.join(OUTPUT_DIR, f'{os.path.splitext(file_name)[0]}.npy')
    
    if os.path.exists(clip_path):
        continue
        
    file_names.append(file_name)

In [9]:
for i in tqdm(range(0, len(file_names), BATCH_SIZE)):
    
    with torch.no_grad():

        images = list()
        names = list()
        for file_name in file_names[i:i+BATCH_SIZE]:
            try:
                image = Image.open(os.path.join(INPUT_DIR, file_name))
                image = preprocessor(images=image, return_tensors="pt")
            except:
                continue
            images.append(image['pixel_values'])
            names.append(file_name)

        images = torch.concat(images, dim=0)
    
        image_features = clip_model.get_image_features(pixel_values=images.to(clip_model.device))
        image_features = image_features.detach().cpu().numpy()
        
        for file_name, image_feature in zip(names, image_features):
            clip_path = os.path.join(OUTPUT_DIR, f'{os.path.splitext(file_name)[0]}.npy')
            np.save(clip_path, image_feature[None, ...])

  0%|          | 0/56 [00:00<?, ?it/s]